# Clustering arrondissement of Paris by their restaurants

## Introduction/Business Problem
#### A description of the problem and a discussion of the background. (15 marks)
Paris is a big city with a lot of historical places, museums and many places to eat. Also, Paris is a multicultural city, where people from different parts of the world live, therefore, there is a wide variety of food offers and from many nationalities. 
It would be really usefull to classify the arrondissements of Paris by the nationality of the food their restaurants offer. This way, as a turist, you would know what kind of food each arrondissement offers and it would be easier to choose a arrondissement to eat.



## Data
#### A description of the data and how it will be used to solve the problem. (15 marks)
I will use the package geopy to get the latitude and longitude of each of the twenty arrondissements. This way, I will create a DataFrame with the necessary information of the arrondissements. This DataFrame will have the number of the arrondissement, and the latitude and longitude of it. Then, with the Foursquare API, I will get the venues of each arrondissement and filter them data to get only information about the restaurants.

The data will be used to clustering the arrondissements and find out which ones offer a similar gastronomic menu.